In [2]:
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/a0/11/9bfcf765e71a2c84bbf715719ba520aeacb2ad84113f14803ff1947ddf69/accelerate-0.28.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/290.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/290.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/290.1 kB ? eta -:--:--
   ----------- --------------------------- 81.9/290.1 kB 919.0 kB/s eta 0:00:01
   ------------------------------------ --- 266.2/290.1 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 290.1/290.1 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


In [3]:
import datasets
import pandas as pd

In [4]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

train_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'train[:20000]')
validation_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'validation')
test_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'test')



Found cached dataset naamapadam (C:/Users/Lenovo/.cache/huggingface/datasets/ai4bharat___naamapadam/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20)
Found cached dataset naamapadam (C:/Users/Lenovo/.cache/huggingface/datasets/ai4bharat___naamapadam/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20)
Found cached dataset naamapadam (C:/Users/Lenovo/.cache/huggingface/datasets/ai4bharat___naamapadam/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20)


In [5]:
features = train_dataset.features
print(features)

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [6]:
text_column_name = "tokens"
label_column_name = "ner_tags"

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [7]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

D:\Study\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\Study\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Tokenize all texts and align the labels with them.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding="max_length",
        truncation=True,
        max_length=256,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    # print(tokenized_inputs)
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    # print(tokenized_inputs)
    return tokenized_inputs

In [9]:
correct_output = tokenize_and_align_labels(train_dataset[0:1])
tokens = tokenizer.convert_ids_to_tokens(correct_output["input_ids"][0])
print(tokens)

# Now we have input ids and labels to pass to model

['[CLS]', '▁सक', 'टर', '▁55', '/', '56', '▁क', '▁एसएचओ', '▁अर', 'वद', '▁कमर', '▁न', '▁बत', 'य', '▁क', '▁इस', '▁म', 'मल', '▁म', '▁आई', 'पस', '▁क', '▁धर', '▁376', '▁-', '▁ड', '▁(', '▁ग', 'गर', 'प', '▁)', '▁क', '▁तहत', '▁म', 'मल', '▁दर', 'ज', '▁कर', '▁लय', '▁ग', 'य', '▁ह', '▁।', '[SEP]', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<p

In [10]:
for token,label in zip(tokenizer.convert_ids_to_tokens(correct_output["input_ids"][0]) , correct_output["labels"][0]):
  print(f"{token:_<40}{label}")


[CLS]___________________________________-100
▁सक_____________________________________0
टर______________________________________-100
▁55_____________________________________0
/_______________________________________-100
56______________________________________-100
▁क______________________________________0
▁एसएचओ__________________________________0
▁अर_____________________________________1
वद______________________________________-100
▁कमर____________________________________2
▁न______________________________________0
▁बत_____________________________________0
य_______________________________________-100
▁क______________________________________0
▁इस_____________________________________0
▁म______________________________________0
मल______________________________________-100
▁म______________________________________0
▁आई_____________________________________0
पस______________________________________-100
▁क______________________________________0
▁धर_____________________________________0
▁376______

In [11]:
tokenized_train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_train_dataset)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})


In [12]:
# Problem is fixed now!!
print(len(tokenized_train_dataset[0]['ner_tags']))
print(len(tokenized_train_dataset[0]['labels']))
print(len(tokenized_train_dataset[0]['input_ids']))

30
256
256


In [13]:
# Test Dataset
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_test_dataset)

Loading cached processed dataset at C:\Users\Lenovo\.cache\huggingface\datasets\ai4bharat___naamapadam\hi\1.0.0\99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20\cache-32ad172ae8f4f3eb.arrow


Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 867
})


In [14]:
# Validation DataSet
tokenized_validation_dataset = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_validation_dataset)

Map:   0%|          | 0/13460 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 13460
})


In [15]:
# Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
# Metric to see how my model is performing For NER we use seqeval
metric = datasets.load_metric("seqeval")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7520\3237337391.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")


In [17]:
# Seeing the working of seqeval
example_text = train_dataset[0]
label_list = train_dataset.features["ner_tags"].feature.names
label_list

labels = [label_list[i] for i in example_text["ner_tags"]]
print(labels)

metric.compute(predictions=[labels] , references=[labels])

['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


{'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [18]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [19]:
# Define Training Args
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
#     weight_decay=0.01,
    save_total_limit=1,
)

In [20]:
trainer = Trainer(
   model,
   args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_validation_dataset,
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

D:\Study\Anaconda\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [ ]:
train_result = trainer.train()
metrics = train_result.metrics

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [76]:
print(metrics)

{'train_runtime': 1090.3439, 'train_samples_per_second': 36.686, 'train_steps_per_second': 4.586, 'total_flos': 441912299520000.0, 'train_loss': 0.29573103637695314, 'epoch': 2.0}


In [77]:
sumoff1score=0.776407+0.624282+0.762044
macrof1=sumoff1score/3
print(macrof1)

0.720911


In [78]:
# Save Model
trainer.save_model("model_indic_ner_5Epoch_one_lakh")

In [79]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    ___
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    ___
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [80]:
zip_dir()

/kaggle/working/directory.zip

In [81]:
for key, value in metrics.items():
    print(f"{key}: {value}")

train_runtime: 1090.3439
train_samples_per_second: 36.686
train_steps_per_second: 4.586
total_flos: 441912299520000.0
train_loss: 0.29573103637695314
epoch: 2.0


# Testing on 20% Testing Dataset

In [ ]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode
lang = 'hi'
test_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'test')
# Test Dataset
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_test_dataset)

In [ ]:
# Evaluation of Model on test Data
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)
trainer.log_metrics("Test",metrics)

sum_of_f1_scores=metrics['test_LOC_f1']+metrics['test_ORG_f1']+metrics['test_PER_f1']
macro_f1=sum_of_f1_scores/3
print('\n Macro f1 score::',macro_f1)